In [1]:
import random
from math import inf


params = {
    "farmer_productivity": 5.0,
    "land_capacity": 80,
    "steal_rate": 2.0,
    "suppression_prob": 0.4,     # p_hit
    "confiscate_rate": 1.5,
    "tax_rate": 0.10,
    "soldier_base_pay": 1.0,

    "alpha_perception": 0.4,
    "beta_tolerance": 0.1,
    "fear_increment": 0.5,
    "fear_decay": 0.2,

    "utility_margin": 0.2,       # margin
    "switch_urge_threshold": 0.5 # urge_threshold
}

EPS = 1e-6
FEAR_MAX = 10.0

class Person:
    def __init__(self, role, wealth=0.0, prod_mult=1.0, aggr_mult=1.0, skill_mult=1.0):
        self.role = role
        self.wealth = max(0.0, wealth)

        self.fear = 0.0
        self.perceived_violence = 0.0
        self.loyalty = 1.0

        self.next_role = role
        self.age_in_role = 0
        self.switch_cooldown = 0  # optional churn control

        # simple heterogeneity
        self.prod_mult = prod_mult
        self.aggr_mult = aggr_mult
        self.skill_mult = skill_mult

    # --- housekeeping ---
    def reset_for_new_tick(self):
        self.next_role = self.role
        self.age_in_role += 1
        if self.switch_cooldown > 0:
            self.switch_cooldown -= 1

    # --- role helpers ---
    def is_farmer(self):  return self.role == "farmer"
    def is_bandit(self):  return self.role == "bandit"
    def is_soldier(self): return self.role == "soldier"

    # --- actions ---
    def produce(self, farmer_productivity, scale=1.0):
        if not self.is_farmer(): return 0.0
        inc = max(0.0, self.prod_mult * farmer_productivity * max(0.0, scale))
        self.wealth = max(0.0, self.wealth + inc)
        return inc

    def attempt_rob(self, victim, steal_rate, fear_bump_robbed=1.0):
        if not (self.is_bandit() and victim and victim.role == "farmer"):
            return 0.0, False
        stolen = min(max(0.0, self.aggr_mult * steal_rate), max(0.0, victim.wealth))
        if stolen <= 0: return 0.0, False
        victim.wealth = max(0.0, victim.wealth - stolen)
        victim.fear = min(FEAR_MAX, victim.fear + max(0.0, fear_bump_robbed))
        self.wealth = max(0.0, self.wealth + stolen)
        return stolen, True

    def attempt_suppress(self, target_bandit, p_hit, confiscate_rate, fear_bump_suppressed=1.0):
        if not (self.is_soldier() and target_bandit and target_bandit.role == "bandit"):
            return 0.0, False
        if random.random() >= max(0.0, min(1.0, self.skill_mult * p_hit)):
            return 0.0, False
        amt = min(max(0.0, self.skill_mult * confiscate_rate), max(0.0, target_bandit.wealth))
        # count as violent even if amt==0 (still coercion)
        target_bandit.wealth = max(0.0, target_bandit.wealth - amt)
        if fear_bump_suppressed > 0:
            target_bandit.fear = min(FEAR_MAX, target_bandit.fear + fear_bump_suppressed)
        self.wealth = max(0.0, self.wealth + amt)
        return amt, True

    def pay_tax(self, tax_rate):
        if not self.is_farmer(): return 0.0
        tax = max(0.0, tax_rate) * max(0.0, self.wealth)
        self.wealth = max(0.0, self.wealth - tax)
        return tax

    def receive_pay(self, amount):
        if amount > 0: self.wealth = max(0.0, self.wealth + amount)

    # --- psychology ---
    def update_psychology(self, global_violence, alpha=0.4, fear_inc=0.5, fear_dec=0.2):
        a = max(0.0, min(1.0, alpha))
        gv = max(0.0, global_violence)
        self.perceived_violence = (1 - a) * self.perceived_violence + a * gv
        if self.perceived_violence > 0.5:
            self.fear = min(FEAR_MAX, self.fear + max(0.0, fear_inc))
        else:
            self.fear = max(0.0, self.fear - max(0.0, fear_dec))

    # --- switching ---
    def decide_switch(self, loyalty_value, utilities, urge_threshold, margin):
        if self.switch_cooldown > 0:
            self.next_role = self.role
            return
        self.loyalty = max(EPS, loyalty_value)
        urge = self.fear / (self.loyalty + EPS)
        if urge <= urge_threshold:
            self.next_role = self.role
            return
        current_u = utilities.get(self.role, -1e9)
        best_role, best_u = self.role, current_u
        for r, u in utilities.items():
            if r != self.role and u > best_u:
                best_role, best_u = r, u
        self.next_role = best_role if best_u > current_u + margin else self.role

    def finalize_role_switch(self):
        if self.next_role != self.role:
            self.role = self.next_role
            self.age_in_role = 0
            self.switch_cooldown = 1  # small hysteresis (optional)

    # --- export ---
    def to_dict(self):
        return {
            "role": self.role,
            "wealth": self.wealth,
            "fear": self.fear,
            "perceived_violence": self.perceived_violence,
            "loyalty": self.loyalty,
            "age_in_role": self.age_in_role,
        }